In [ ]:
import numpy as np
import pandas as pd
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
from tqdm import tqdm

# Load Fashion MNIST dataset
def load_fashion_mnist():
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    x_train = x_train / 255.0
    x_test = x_test / 255.0
    return (x_train, y_train), (x_test, y_test)

# Display sample images
def display_sample_images(x_train, y_train, label):
    print_Once = [1] * 10
    count = 10
    for i in range(60000):
        if count == 0:
            break
        if print_Once[y_train[i]]:
            print_Once[y_train[i]] -= 1
            count -= 1
            plt.figure(figsize=(1, 1))
            plt.imshow(x_train[i], cmap="Greys")
            plt.title("{}".format(label[y_train[i]]))
            plt.xticks([])
            plt.yticks([])
            plt.show()

# Initialize weights and biases
def initialize_weights(initialization_func, prev_layer_neurons, no_of_hidden_layers, classes):
    theta = []
    for i in range(2 * (no_of_hidden_layers + 1)):
        theta.append([])
    for i in range(no_of_hidden_layers):
        neurons_in_layer = int(input(f"Number of neurons in layer {i}: "))
        make_weights(theta, neurons_in_layer, prev_layer_neurons, i, initialization_func)
        make_biases(theta, neurons_in_layer, no_of_hidden_layers + 1 + i, initialization_func)
        prev_layer_neurons = neurons_in_layer
    make_weights(theta, classes, prev_layer_neurons, no_of_hidden_layers, initialization_func)
    make_biases(theta, classes, 2 * (no_of_hidden_layers + 1) - 1, initialization_func)
    return theta

# Make weights
def make_weights(theta, curr_layer_neurons, prev_layer_neurons, layer_no, initialization_func):
    if initialization_func == "random":
        theta[layer_no] = np.float128(np.random.randn(curr_layer_neurons, prev_layer_neurons) * 0.01)
    elif initialization_func == "Xavier":
        factor = np.sqrt(6.0 / (curr_layer_neurons + prev_layer_neurons))
        theta[layer_no] = np.float128(np.random.uniform(low=-factor, high=factor, size=(curr_layer_neurons, prev_layer_neurons)))

# Make biases
def make_biases(theta, curr_layer_neurons, layer_no, initialization_func):
    if initialization_func == "random":
        theta[layer_no] = np.float128(np.random.randn(curr_layer_neurons, 1) * 0.01)
    elif initialization_func == "Xavier":
        theta[layer_no] = np.float128(np.zeros((curr_layer_neurons, 1)))

# Calculate activation function
def calc_activation(a, activation_func):
    h = []
    a=np.round(a,6)
    for i in range(len(a)):
        if activation_func == "sigmoid":
          if(a[i][0]<-100):
            h.append(0.0)
          else:
            h.append(1/(1+np.exp(-a[i][0])))

        elif activation_func == "ReLU":
            h.append(max(0, a[i][0]))

        elif activation_func == "tanh":
           h.append(2 * (1 / (1 + np.exp(-2 * a[i][0]))) - 1)
    h = np.array(h)
    h_new = h.reshape((len(h), 1))
    return h_new

# Calculate activation derivative
def calc_activation_derivative(a, activation_func):
    if activation_func == "sigmoid":
        a=0.0 if(a<-100) else 1/(1+np.exp(a))
        return a * (1 - (a))
    elif activation_func == "ReLU":
        return np.where(a > 0, 1, 0)
    elif activation_func == "tanh":
        return 1/(np.cosh(a)**2)

def calc_gdash(ak,activationFunc):
  gdsh=[]
  for i in ak:
      gdsh.append(calc_activation_derivative(i[0],activationFunc))
  gdsh=np.array(gdsh)
  gdshNew=gdsh.reshape((len(gdsh),1))
  return gdshNew

def calc_aL(aL,y):
  aL[y][0]=-(1-aL[y][0])
  return aL

# Calculate softmax
def calc_softmax(a):
    #return np.exp(a) / np.sum(np.exp(a), axis=0)
    exp_a = np.exp(a - np.max(a, axis=0))
    return exp_a / np.sum(exp_a, axis=0)

# Forward propagation
def forward_propagation(theta, inp_list, activation_func):
    a_h_list = []
    h = inp_list
    for i in range(no_of_hidden_layers):
        a = np.dot(theta[i], h) + theta[no_of_hidden_layers + 1 + i]
        a_h_list.append(a)
        h = calc_activation(a, activation_func)
        a_h_list.append(h)
    a = np.dot(theta[no_of_hidden_layers], h) + theta[-1]
    a_h_list.append(a)
    y_hat = calc_softmax(a)
    a_h_list.append(y_hat)
    return a_h_list

# Calculate loss
def calc_loss(prediction, actual, loss_type):
    if loss_type == "mse":
        return 0.5 * np.mean(np.square(prediction - actual))
    elif loss_type == "cross_entropy":
        prediction = np.clip(prediction, 1e-10, 1.0 - 1e-10)
        return -np.log(prediction)

def calc_val_loss_acc(theta,validation_split,activation_func,loss_type):
      correct = 0
      total = int(60000 * validation_split)
      validation_loss = 0.0
      for i in range(59999, 59999 - total - 1, -1):
          a_h_list = forward_propagation(theta, x_train[i].flatten().reshape((784, 1)), activation_func)
          prediction = np.argmax(a_h_list[-1])
          if prediction == y_train[i]:
              correct += 1
          validation_loss += calc_loss(prediction, y_train[i], loss_type)
      validation_accuracy = correct / total
      validation_loss /= total
      return validation_accuracy,validation_loss

# Back propagation
def back_propagation(a_h_list, y, inp, del_theta, theta, batch_size, activation_func):
    h_counter = len(a_h_list) - 1
    grad_a = calc_aL(a_h_list[h_counter],y)
    h_counter -= 2
    for i in range(no_of_hidden_layers, -1, -1):
        if i == 0:
            del_w = np.dot(grad_a, inp.T)
            del_b = grad_a
            del_theta[i] = np.add(del_theta[i], del_w)
            del_theta[i + no_of_hidden_layers + 1] = np.add(del_theta[i + no_of_hidden_layers + 1], del_b)
            break
        del_w = np.dot(grad_a, a_h_list[h_counter].T)
        del_b = grad_a
        del_theta[i] = np.add(del_theta[i], del_w)
        del_theta[i + no_of_hidden_layers + 1] = np.add(del_theta[i + no_of_hidden_layers + 1], del_b)
        grad_h_prev = np.dot(theta[i].T, grad_a)
        grad_a = grad_h_prev * calc_gdash(a_h_list[h_counter - 1], activation_func)
        h_counter -= 2

# Gradient Descent
def gradient_descent(eta, batch_size, epoch, theta, activation_func, validation_split, loss_type, alpha):
    for itr in range(epoch):
        # Initialize gradients and total loss
        del_theta = [np.zeros_like(param) for param in theta]
        total_loss = 0
        total_acc = 0

        # Iterate through the training data
        for i in tqdm(range(int(60000 * (1 - validation_split)))):
            # Forward propagation
            a_h_list = forward_propagation(theta, np.float128(x_train[i].flatten().reshape((784, 1))), activation_func)

            # Compute loss
            total_loss += calc_loss(np.argmax(a_h_list[-1]), y_train[i], loss_type)

            # Update accuracy
            if np.argmax(a_h_list[-1]) == y_train[i]:
                total_acc += 1

            # Backpropagation
            back_propagation(a_h_list, y_train[i], np.float128((x_train[i].flatten()).reshape((784, 1))), del_theta, theta, batch_size, activation_func)

            # Update weights after every mini-batch
            if i % batch_size == 0 and i != 0:
                for j in range(len(theta)):
                    del_theta[j] = (del_theta[j] / batch_size)+alpha*theta[j]
                    #temp=np.subtract(theta[j],eta*del_theta[j])
                    #norm=np.linalg.norm(temp)
                    #theta[j]=temp/norm
                    theta[j] = np.subtract(theta[j], eta * del_theta[j])
                    del_theta[j] = del_theta[j] * 0

        # Calculate validation loss and accuracy
        validation_accuracy,validation_loss=calc_val_loss_acc(theta,validation_split,activation_func,loss_type)

        # Print epoch statistics
        print(f"Epoch {itr + 1}/{epoch}, Loss: {total_loss / (60000 * (1 - validation_split))}, Accuracy: {total_acc / (60000 * (1 - validation_split))}, Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

# Momentum Gradient Descent
def momentum_gradient_descent(eta, batch_size, epoch, theta, beta, activationFunc, validation_split, loss_type, alpha):
    # Initialize previous history
    prev_history = [np.zeros_like(param) for param in theta]

    for itr in range(epoch):
        total_loss = 0
        total_acc = 0
        del_theta = [np.zeros_like(param) for param in theta]

        # Iterate through the training data
        for i in tqdm(range(int(60000 * (1 - validation_split)))):
            # Forward propagation
            a_h_list = forward_propagation(theta, np.float128(x_train[i].flatten().reshape((784, 1))), activationFunc)

            # Compute loss
            total_loss += calc_loss(np.argmax(a_h_list[-1]), y_train[i], loss_type)

            # Update accuracy
            if np.argmax(a_h_list[-1]) == y_train[i]:
                total_acc += 1

            # Backpropagation
            back_propagation(a_h_list, y_train[i], np.float128((x_train[i].flatten()).reshape((784, 1))), del_theta, theta, batch_size, activationFunc)

            # Update weights using momentum
            if i % batch_size == 0 and i != 0:
                for j in range(len(del_theta)):
                    del_theta[j] = (del_theta[j] / batch_size)+alpha*theta[j]
                    prev_history[j] = np.add(beta * prev_history[j],eta * (del_theta[j]))
                    theta[j] = np.subtract(theta[j], eta * prev_history[j])
                    del_theta[j] = del_theta[j] * 0

        # Calculate validation loss and accuracy
        validation_accuracy,validation_loss=calc_val_loss_acc(theta,validation_split,activation_func,loss_type)

        # Print epoch statistics
        print(f"Epoch {itr + 1}/{epoch}, Loss: {total_loss / (60000 * (1 - validation_split))}, Accuracy: {total_acc / (60000 * (1 - validation_split))}, Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

# Nestrov Gradient Descent
def nesterov_gradient_descent(eta, batch_size, epoch, theta, beta, activationFunc, validation_split, loss_type, alpha):
    # Initialize previous history
    prev_history = [np.zeros_like(param) for param in theta]

    for itr in range(epoch):
        total_loss = 0
        total_acc = 0
        del_theta = [np.zeros_like(param) for param in theta]

        # Iterate through the training data
        for i in tqdm(range(int(60000 * (1 - validation_split)))):
            # Update weights using Nesterov accelerated gradient descent
            updated_theta = [theta[j] - beta * prev_history[j] for j in range(len(theta))]

            # Forward propagation
            a_h_list = forward_propagation(updated_theta, np.float128(x_train[i].flatten().reshape((784, 1))), activationFunc)

            # Compute loss
            total_loss += calc_loss(np.argmax(a_h_list[-1]), y_train[i], loss_type)

            # Update accuracy
            if np.argmax(a_h_list[-1]) == y_train[i]:
                total_acc += 1

            # Backpropagation
            back_propagation(a_h_list, y_train[i], np.float128((x_train[i].flatten()).reshape((784, 1))), del_theta, updated_theta, batch_size, activationFunc)

            # Update weights using momentum
            if i % batch_size == 0 and i != 0:
                for j in range(len(del_theta)):
                    del_theta[j] = (del_theta[j] / batch_size)+alpha*theta[j]
                    prev_history[j] = np.add(beta * prev_history[j],eta*(del_theta[j]))
                    theta[j] = np.subtract(theta[j], prev_history[j])
                    del_theta[j] = del_theta[j] * 0

        # Calculate validation loss and accuracy
        validation_accuracy,validation_loss=calc_val_loss_acc(theta,validation_split,activation_func,loss_type)

        # Print epoch statistics
        print(f"Epoch {itr + 1}/{epoch}, Loss: {total_loss / (60000 * (1 - validation_split))}, Accuracy: {total_acc / (60000 * (1 - validation_split))}, Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

# RMS_Prop
def rmsprop(eta, batch_size, epoch, theta, beta, eps, activation_func, validation_split, loss_type, alpha):
    # Initialize first  moment estimates
    v_theta = [np.zeros_like(param) for param in theta]

    for itr in range(epoch):
        total_loss = 0
        total_acc = 0
        del_theta = [np.zeros_like(param) for param in theta]

        # Iterate through the training data
        for i in tqdm(range(int(60000 * (1 - validation_split)))):
            # Forward propagation
            a_h_list = forward_propagation(theta, np.float128(x_train[i].flatten().reshape((784, 1))), activation_func)

            # Compute loss
            total_loss += calc_loss(np.argmax(a_h_list[-1]), y_train[i], loss_type)

            # Update accuracy
            if np.argmax(a_h_list[-1]) == y_train[i]:
                total_acc += 1

            # Backpropagation
            back_propagation(a_h_list, y_train[i], np.float128((x_train[i].flatten()).reshape((784, 1))), del_theta, theta, batch_size, activation_func)

            # Update weights using Adam optimizer
            if i % batch_size == 0 and i != 0:
                for j in range(len(theta)):
                    del_theta[j] = (del_theta[j] / batch_size)+alpha*theta[j]
                    v_theta[j] = beta * v_theta[j] + (1 - beta) * (del_theta[j] ** 2)

                    # Update weights
                    theta[j] = np.subtract(theta[j], eta * (del_theta[j]/ (np.sqrt(v_theta[j] + eps))))

         # Calculate validation loss and accuracy
        validation_accuracy,validation_loss=calc_val_loss_acc(theta,validation_split,activation_func,loss_type)

        # Print epoch statistics
        print(f"Epoch {itr + 1}/{epoch}, Loss: {total_loss / (60000 * (1 - validation_split))}, Accuracy: {total_acc / (60000 * (1 - validation_split))}, Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")


# Adam Optimizer
def adam_optimizer(eta, batch_size, epoch, theta, beta1, beta2, eps, activation_func, validation_split, loss_type, alpha):
    # Initialize first and second moment estimates
    m_theta = [np.zeros_like(param) for param in theta]
    v_theta = [np.zeros_like(param) for param in theta]

    for itr in range(epoch):
        total_loss = 0
        total_acc = 0
        del_theta = [np.zeros_like(param) for param in theta]

        # Iterate through the training data
        for i in tqdm(range(int(60000 * (1 - validation_split)))):
            # Forward propagation
            a_h_list = forward_propagation(theta, np.float128(x_train[i].flatten().reshape((784, 1))), activation_func)

            # Compute loss
            total_loss += calc_loss(np.argmax(a_h_list[-1]), y_train[i], loss_type)

            # Update accuracy
            if np.argmax(a_h_list[-1]) == y_train[i]:
                total_acc += 1

            # Backpropagation
            back_propagation(a_h_list, y_train[i], np.float128((x_train[i].flatten()).reshape((784, 1))), del_theta, theta, batch_size, activation_func)

            # Update weights using Adam optimizer
            if i % batch_size == 0 and i != 0:
                for j in range(len(theta)):
                    del_theta[j] = (del_theta[j] / batch_size)+alpha*theta[j]
                    m_theta[j] = beta1 * m_theta[j] + (1 - beta1) * del_theta[j]
                    v_theta[j] = beta2 * v_theta[j] + (1 - beta2) * (del_theta[j] ** 2)

                    # Bias correction
                    m_hat = m_theta[j] / (1 - np.power(beta1, itr + 1))
                    v_hat = v_theta[j] / (1 - np.power(beta2, itr + 1))

                    # Update weights
                    theta[j] = np.subtract(theta[j], eta * m_hat / (np.sqrt(v_hat) + eps))

         # Calculate validation loss and accuracy
        validation_accuracy,validation_loss=calc_val_loss_acc(theta,validation_split,activation_func,loss_type)

        # Print epoch statistics
        print(f"Epoch {itr + 1}/{epoch}, Loss: {total_loss / (60000 * (1 - validation_split))}, Accuracy: {total_acc / (60000 * (1 - validation_split))}, Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

# nadam Optimizer
def nadam_optimizer(eta, batch_size, epoch, theta, beta1, beta2, eps, activation_func, validation_split, loss_type, alpha):
    # Initialize first and second moment estimates
    m_theta = [np.zeros_like(param) for param in theta]
    v_theta = [np.zeros_like(param) for param in theta]

    for itr in range(epoch):
        total_loss = 0
        total_acc = 0
        del_theta = [np.zeros_like(param) for param in theta]

        # Iterate through the training data
        for i in tqdm(range(int(60000 * (1 - validation_split)))):
            # Forward propagation
            a_h_list = forward_propagation(theta, np.float128(x_train[i].flatten().reshape((784, 1))), activation_func)

            # Compute loss
            total_loss += calc_loss(np.argmax(a_h_list[-1]), y_train[i], loss_type)

            # Update accuracy
            if np.argmax(a_h_list[-1]) == y_train[i]:
                total_acc += 1

            # Backpropagation
            back_propagation(a_h_list, y_train[i], np.float128((x_train[i].flatten()).reshape((784, 1))), del_theta, theta, batch_size, activation_func)

            # Update weights using Adam optimizer
            if i % batch_size == 0 and i != 0:
                for j in range(len(theta)):
                    del_theta[j] = (del_theta[j] / batch_size)+alpha*theta[j]
                    m_theta[j] = beta1 * m_theta[j] + (1 - beta1) * del_theta[j]
                    v_theta[j] = beta2 * v_theta[j] + (1 - beta2) * (del_theta[j] ** 2)

                    # Bias correction
                    m_hat = m_theta[j] / (1 - np.power(beta1, itr + 1))
                    v_hat = v_theta[j] / (1 - np.power(beta2, itr + 1))

                    # Update weights
                    theta[j] = np.subtract(theta[j], (eta / (np.sqrt(v_hat) + eps))*((beta1*m_hat)+((1-beta1)*(del_theta[j]/(1 - np.power(beta1, itr + 1))))))

         # Calculate validation loss and accuracy
        validation_accuracy,validation_loss=calc_val_loss_acc(theta,validation_split,activation_func,loss_type)

        # Print epoch statistics
        print(f"Epoch {itr + 1}/{epoch}, Loss: {total_loss / (60000 * (1 - validation_split))}, Accuracy: {total_acc / (60000 * (1 - validation_split))}, Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")


# Main code
(x_train, y_train), (x_test, y_test) = load_fashion_mnist()
label = {0: "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat", 5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle boot"}
no_of_hidden_layers = int(input("Enter the number of hidden layers: "))
theta = initialize_weights("random", 784, no_of_hidden_layers, 10)
eta = 1e-2
batch_size = 32
epoch = 5
beta=0.9
beta1 = 0.9
beta2 = 0.999
eps = 1e-10
alpha=0.0005
validation_split = 0.1
activation_func = "sigmoid"
loss_type = "cross_entropy"
gradient_descent(eta,1,epoch,theta,activation_func,validation_split,loss_type,alpha)
#momentum_gradient_descent(eta,batch_size,epoch,theta,beta,activation_func,validation_split,loss_type,alpha)
#nesterov_gradient_descent(eta,batch_size,epoch,theta,beta,activation_func,validation_split,loss_type,alpha)
#rmsprop(eta,batch_size,epoch,theta,beta,eps,activation_func,validation_split,loss_type,alpha)
#adam_optimizer(eta,batch_size,epoch,theta,beta1,beta2,eps,activation_func,validation_split,loss_type,alpha)
#nadam_optimizer(eta,batch_size,epoch,theta,beta1,beta2,eps,activation_func,validation_split,loss_type,alpha)